# Market Sniper Screener v2.0

**Improved stock screener with:**
- Fixed technical indicator calculations (RSI, ADX)
- Volume confirmation for signals
- Trend confirmation (SMA slope + price structure)
- Relative strength vs SPY
- Flexible VIX handling

Run each cell in order. Configuration can be adjusted in Cell 2.

In [ ]:
# Cell 1: Install dependencies
!pip install yfinance pandas numpy -q
print("Dependencies installed!")

In [ ]:
# Cell 2: Configuration - ADJUST THESE SETTINGS

# === RISK MANAGEMENT ===
STOP_LOSS_PCT = 12.0          # Stop loss percentage
TAKE_PROFIT_PCT = 36.0        # Take profit (3:1 reward/risk)

# === TRAILING STOP ===
USE_TRAILING = True           # Enable trailing stop
TRAIL_ACTIVATION_PCT = 12.0   # Activate after this gain (1R)
TRAIL_DISTANCE_PCT = 8.0      # Trail this far below highest high

# === TIME EXIT ===
MAX_HOLD_DAYS = 90            # Maximum holding period

# === VIX FILTER ===
USE_VIX_FILTER = True         # Set False to disable VIX filter
VIX_MIN = 18                  # Minimum VIX (elevated fear)
VIX_MAX = 40                  # Maximum VIX (avoid extreme panic)

# === VOLUME FILTER ===
USE_VOLUME_FILTER = True      # Require above-average volume
VOLUME_SURGE_MULT = 1.3       # Volume must be 1.3x average

# === RELATIVE STRENGTH ===
USE_RS_FILTER = True          # Compare performance to SPY

# === POSITION SIZING ===
ACCOUNT_SIZE = 100000         # Account size for position sizing
RISK_PER_TRADE_PCT = 1.0      # Risk per trade (1% of account)

# === DATE RANGE ===
START_DATE = '2018-01-01'
# END_DATE = '2024-01-01'  # Uncomment to set custom end date

print("Configuration loaded!")
print(f"  Stop Loss: {STOP_LOSS_PCT}%")
print(f"  Take Profit: {TAKE_PROFIT_PCT}% ({TAKE_PROFIT_PCT/STOP_LOSS_PCT:.1f}R)")
print(f"  Trailing: {'Enabled' if USE_TRAILING else 'Disabled'}")
print(f"  VIX Filter: {'Enabled' if USE_VIX_FILTER else 'Disabled'}")
print(f"  Volume Filter: {'Enabled' if USE_VOLUME_FILTER else 'Disabled'}")

In [ ]:
# Cell 3: Imports and setup
import pandas as pd
import numpy as np
import yfinance as yf
import time
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
import warnings
warnings.filterwarnings('ignore')

# Set end date if not defined
try:
    END_DATE
except NameError:
    END_DATE = datetime.now().strftime('%Y-%m-%d')

# Additional config
MIN_MARKET_CAP = 1e9
MIN_BARS = 500
RSI_OVERSOLD = 35
RSI_SIGNAL = 45
RSI_LOOKBACK = 5
ADX_MIN = 20
MIN_BELOW_HIGH_PCT = 25.0
MAX_BELOW_HIGH_PCT = 50.0
MAX_FROM_SMA_PCT = 12.0
SMA_SLOPE_DAYS = 20
VOLUME_AVG_DAYS = 50
RS_LOOKBACK = 63
MAX_PE = 60
MAX_PEG = 3.0
MAX_DEBT_EQUITY = 3.0
MIN_PROFIT_MARGIN = -0.3
MIN_FUNDAMENTAL_CHECKS = 1
MIN_GAP_DAYS = 30
MAX_POSITION_PCT = 15.0

print("Setup complete!")

In [ ]:
# Cell 4: Ticker Universe
TICKERS = [
    # S&P 500 Core
    'AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEP',
    'AFL', 'AIG', 'AMAT', 'AMD', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'AON',
    'APD', 'APH', 'AVGO', 'AXP', 'AZO', 'BA', 'BAC', 'BDX', 'BIIB', 'BK',
    'BKNG', 'BLK', 'BMY', 'BSX', 'C', 'CAT', 'CB', 'CCI', 'CDNS', 'CEG',
    'CHTR', 'CI', 'CL', 'CMCSA', 'CME', 'CMG', 'CNC', 'COF', 'COP', 'COST',
    'CRM', 'CSCO', 'CSX', 'CTAS', 'CVS', 'CVX', 'D', 'DD', 'DE', 'DHR',
    'DIS', 'DLR', 'DOV', 'DOW', 'DUK', 'ECL', 'EL', 'ELV', 'EMR', 'EOG',
    'EQIX', 'ETN', 'EW', 'EXC', 'F', 'FANG', 'FCX', 'FDX', 'FI', 'FICO',
    'FIS', 'FISV', 'GD', 'GE', 'GEHC', 'GILD', 'GIS', 'GLW', 'GM', 'GOOG',
    'GOOGL', 'GPN', 'GS', 'GWW', 'HAL', 'HCA', 'HD', 'HLT', 'HON', 'HPQ',
    'HUM', 'IBM', 'ICE', 'IDXX', 'INTC', 'INTU', 'ISRG', 'ITW', 'JCI', 'JNJ',
    'JPM', 'KDP', 'KEY', 'KLAC', 'KMB', 'KO', 'KR', 'LEN', 'LHX', 'LIN',
    'LLY', 'LMT', 'LOW', 'LRCX', 'LULU', 'LUV', 'MA', 'MAR', 'MCD', 'MCHP',
    'MCK', 'MCO', 'MDLZ', 'MDT', 'MET', 'META', 'MMC', 'MMM', 'MNST', 'MO',
    'MPC', 'MRK', 'MRNA', 'MS', 'MSCI', 'MSFT', 'MSI', 'MTB', 'MU', 'NDAQ',
    'NEE', 'NFLX', 'NKE', 'NOC', 'NOW', 'NSC', 'NTRS', 'NUE', 'NVDA', 'NVR',
    'ORCL', 'ORLY', 'OXY', 'PANW', 'PAYX', 'PCAR', 'PEP', 'PFE', 'PG', 'PGR',
    'PH', 'PLD', 'PM', 'PNC', 'PPG', 'PRU', 'PSA', 'PSX', 'PYPL', 'QCOM',
    'REGN', 'RJF', 'RMD', 'ROK', 'ROP', 'ROST', 'RSG', 'RTX', 'SBAC', 'SBUX',
    'SCHW', 'SHW', 'SLB', 'SNPS', 'SO', 'SPG', 'SPGI', 'SRE', 'STE', 'STZ',
    'SYK', 'SYY', 'T', 'TDG', 'TEL', 'TFC', 'TGT', 'TJX', 'TMO', 'TMUS',
    'TROW', 'TRV', 'TSLA', 'TT', 'TXN', 'TYL', 'UNH', 'UNP', 'UPS', 'URI',
    'USB', 'V', 'VLO', 'VMC', 'VRSN', 'VRTX', 'VZ', 'WAB', 'WBA', 'WBD',
    'WELL', 'WFC', 'WM', 'WMB', 'WMT', 'WRB', 'XEL', 'XOM', 'XYL', 'YUM',
    'ZBH', 'ZTS',
    # High Growth / Tech
    'ABNB', 'AFRM', 'AI', 'BILL', 'COIN', 'CRWD', 'DDOG', 'DOCU', 'ESTC',
    'FTNT', 'GLOB', 'GTLB', 'HUBS', 'MDB', 'MNDY', 'NET', 'NTNX', 'OKTA',
    'PCTY', 'PLTR', 'RBLX', 'ROKU', 'SHOP', 'SNAP', 'SNOW', 'SOFI', 'SPOT',
    'SQ', 'TTD', 'TWLO', 'U', 'UBER', 'UPST', 'VEEV', 'WIX', 'ZI', 'ZM', 'ZS',
    # Biotech
    'ALNY', 'ARGX', 'BGNE', 'BNTX', 'CRSP', 'DXCM', 'EXAS', 'INCY', 'IONS',
    'JAZZ', 'LEGN', 'MDGL', 'NBIX', 'NTLA', 'RARE', 'RPRX', 'SRPT', 'UTHR',
    'VCEL', 'VKTX', 'XENE',
    # Semiconductors
    'ALGN', 'ASML', 'CRUS', 'ENPH', 'FSLR', 'LSCC', 'MRVL', 'MPWR', 'NXPI',
    'ON', 'QRVO', 'SEDG', 'SMCI', 'SWKS', 'TER', 'WOLF',
    # Energy
    'AM', 'APA', 'AR', 'BKR', 'CHK', 'CHRD', 'CNX', 'CTRA', 'DVN', 'EQT',
    'HES', 'KMI', 'MRO', 'MTDR', 'NOV', 'OKE', 'OVV', 'PXD', 'RRC',
    'SWN', 'TRGP',
    # Financials
    'ACGL', 'AFG', 'AJG', 'ALL', 'ALLY', 'APO', 'ARES', 'AXS', 'BRO',
    'BX', 'CFG', 'CG', 'CINF', 'DFS', 'ERIE', 'EVR', 'FNB', 'HBAN', 'HIG',
    'IBKR', 'KKR', 'L', 'LPLA', 'MTG', 'NMIH', 'ORI', 'PFG', 'PIPR', 'RDN',
    'RGA', 'RNR', 'SEIC', 'SF', 'SLM', 'STT', 'UNM', 'VOYA', 'WRB',
    # Industrials
    'AGCO', 'AXON', 'BLDR', 'CMI', 'EME', 'FAST', 'GNRC', 'HEI', 'HWM',
    'IEX', 'IR', 'ITT', 'J', 'JBHT', 'JBL', 'KBR', 'LDOS', 'MAS', 'MLI',
    'MOD', 'ODFL', 'OSK', 'PWR', 'RBC', 'SAIA', 'SNA', 'STRL', 'TDY',
    'TTC', 'UFPI', 'WCC', 'XPO',
    # Consumer
    'AEO', 'ANF', 'BURL', 'CHWY', 'CPRI', 'CROX', 'CVNA', 'DASH',
    'DECK', 'DKS', 'DPZ', 'ETSY', 'FIVE', 'FND', 'GPS', 'HAS', 'KMX', 'LVS',
    'LYFT', 'MAT', 'NCLH', 'OLLI', 'RCL', 'RH', 'SHAK', 'SKX', 'TPR',
    'TRIP', 'UA', 'ULTA', 'URBN', 'W', 'WING', 'WSM', 'WYNN',
    # Materials
    'AA', 'ALB', 'BALL', 'CCJ', 'CF', 'CLF', 'EMN', 'FMC', 'GOLD', 'IFF',
    'IP', 'KGC', 'MLM', 'MOS', 'MP', 'NEM', 'NTR', 'PKG', 'RS', 'SCCO',
    'SMG', 'STLD', 'TECK', 'VALE', 'WPM',
    # EV / Clean Energy
    'CHPT', 'LAC', 'LCID', 'LI', 'NIO', 'PLUG', 'QS', 'RIVN', 'RUN', 'XPEV',
    # Crypto-adjacent
    'MARA', 'MSTR', 'RIOT',
]

TICKERS = sorted(list(set(TICKERS)))
print(f"Loaded {len(TICKERS)} unique tickers")

In [ ]:
# Cell 5: Technical Indicators (Fixed Implementations)

def calc_rsi(close: pd.Series, length: int = 14) -> pd.Series:
    """Calculate RSI using Wilder's smoothing"""
    delta = close.diff()
    gain = delta.where(delta > 0, 0.0)
    loss = (-delta).where(delta < 0, 0.0)
    avg_gain = gain.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi.fillna(50)

def calc_adx(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 14):
    """Calculate ADX, +DI, -DI (fixed implementation)"""
    prev_close = close.shift(1)
    tr1 = high - low
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    up_move = high - high.shift(1)
    down_move = low.shift(1) - low
    
    plus_dm = pd.Series(np.where((up_move > down_move) & (up_move > 0), up_move, 0), index=high.index)
    minus_dm = pd.Series(np.where((down_move > up_move) & (down_move > 0), down_move, 0), index=high.index)
    
    atr = tr.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
    plus_dm_smooth = plus_dm.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
    minus_dm_smooth = minus_dm.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
    
    plus_di = 100 * plus_dm_smooth / atr.replace(0, np.nan)
    minus_di = 100 * minus_dm_smooth / atr.replace(0, np.nan)
    
    di_sum = plus_di + minus_di
    di_diff = (plus_di - minus_di).abs()
    dx = 100 * di_diff / di_sum.replace(0, np.nan)
    adx = dx.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
    
    return adx.fillna(0), plus_di.fillna(0), minus_di.fillna(0)

def calc_sma(series: pd.Series, length: int) -> pd.Series:
    return series.rolling(window=length, min_periods=length).mean()

def calc_sma_slope(sma: pd.Series, lookback: int = 20) -> pd.Series:
    return (sma - sma.shift(lookback)) / sma.shift(lookback) * 100

def calc_relative_strength(stock_close: pd.Series, benchmark_close: pd.Series, lookback: int = 63) -> pd.Series:
    stock_ret = stock_close.pct_change(lookback)
    bench_ret = benchmark_close.pct_change(lookback)
    return stock_ret - bench_ret

print("Indicator functions loaded!")

In [ ]:
# Cell 6: Data Fetching Functions

def get_vix_data(start_date, end_date):
    print("Downloading VIX data...")
    try:
        vix = yf.download('^VIX', start=start_date, end=end_date, progress=False)
        if isinstance(vix.columns, pd.MultiIndex):
            vix.columns = [col[0] for col in vix.columns]
        vix_lookup = {date.strftime('%Y-%m-%d'): float(row['Close']) for date, row in vix.iterrows()}
        if USE_VIX_FILTER:
            in_range = sum(1 for v in vix_lookup.values() if VIX_MIN <= v <= VIX_MAX)
            print(f"  VIX in range [{VIX_MIN}-{VIX_MAX}]: {in_range} days ({in_range/len(vix_lookup)*100:.1f}%)")
        return vix_lookup
    except Exception as e:
        print(f"Warning: Could not download VIX: {e}")
        return {}

def get_spy_data(start_date, end_date):
    print("Downloading SPY benchmark...")
    try:
        spy = yf.download('SPY', start=start_date, end=end_date, progress=False)
        if isinstance(spy.columns, pd.MultiIndex):
            spy.columns = [col[0] for col in spy.columns]
        return spy['Close']
    except:
        return None

def get_stock_data(ticker, start_date, end_date):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        market_cap = info.get('marketCap', 0) or 0
        if market_cap < MIN_MARKET_CAP:
            return None, None
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if df is None or len(df) < MIN_BARS:
            return None, None
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = [col[0] for col in df.columns]
        if df['Close'].isna().sum() > len(df) * 0.05:
            return None, None
        df = df.ffill(limit=3)
        return df, info
    except:
        return None, None

def check_fundamentals(info):
    checks = []
    pe = info.get('trailingPE') or info.get('forwardPE')
    if pe is not None:
        checks.append(0 < pe < MAX_PE)
    peg = info.get('pegRatio')
    if peg is not None:
        checks.append(0 < peg < MAX_PEG)
    de = info.get('debtToEquity')
    if de is not None:
        de = de / 100 if de > 10 else de
        checks.append(de < MAX_DEBT_EQUITY)
    pm = info.get('profitMargins')
    if pm is not None:
        checks.append(pm > MIN_PROFIT_MARGIN)
    return sum(checks), len(checks)

print("Data functions loaded!")

In [ ]:
# Cell 7: Trade Execution

def execute_trade(close, high, low, entry_idx, entry_price):
    """Execute trade with stops and targets"""
    stop_price = entry_price * (1 - STOP_LOSS_PCT / 100)
    tp_price = entry_price * (1 + TAKE_PROFIT_PCT / 100)
    activation_price = entry_price * (1 + TRAIL_ACTIVATION_PCT / 100)
    
    highest_high = entry_price
    trailing_active = False
    trailing_stop = stop_price
    max_dd, max_runup = 0.0, 0.0
    
    for day in range(1, MAX_HOLD_DAYS + 1):
        idx = entry_idx + day
        if idx >= len(close):
            exit_price = close[-1]
            return {
                'return_pct': (exit_price - entry_price) / entry_price * 100,
                'exit_price': exit_price, 'exit_day': day, 'exit_reason': 'data_end',
                'max_dd': max_dd, 'max_runup': max_runup, 'trailing_activated': trailing_active
            }
        
        day_high, day_low = high[idx], low[idx]
        low_ret = (day_low - entry_price) / entry_price * 100
        high_ret = (day_high - entry_price) / entry_price * 100
        max_dd, max_runup = min(max_dd, low_ret), max(max_runup, high_ret)
        
        if day_high > highest_high:
            highest_high = day_high
        
        if USE_TRAILING and not trailing_active and day_high >= activation_price:
            trailing_active = True
        
        if trailing_active:
            new_trail = highest_high * (1 - TRAIL_DISTANCE_PCT / 100)
            trailing_stop = max(trailing_stop, new_trail)
        
        current_stop = trailing_stop if trailing_active else stop_price
        
        if day_low <= current_stop:
            return {
                'return_pct': (current_stop - entry_price) / entry_price * 100,
                'exit_price': current_stop, 'exit_day': day,
                'exit_reason': 'trail_stop' if trailing_active else 'stop',
                'max_dd': max_dd, 'max_runup': max_runup, 'trailing_activated': trailing_active
            }
        
        if day_high >= tp_price:
            return {
                'return_pct': TAKE_PROFIT_PCT, 'exit_price': tp_price, 'exit_day': day,
                'exit_reason': 'target', 'max_dd': max_dd, 'max_runup': max_runup,
                'trailing_activated': trailing_active
            }
    
    exit_price = close[entry_idx + MAX_HOLD_DAYS]
    return {
        'return_pct': (exit_price - entry_price) / entry_price * 100,
        'exit_price': exit_price, 'exit_day': MAX_HOLD_DAYS, 'exit_reason': 'max_days',
        'max_dd': max_dd, 'max_runup': max_runup, 'trailing_activated': trailing_active
    }

print("Trade execution loaded!")

In [ ]:
# Cell 8: Signal Generation

def check_entry_signal(idx, close, high, low, volume, rsi, adx, plus_di, minus_di,
                       sma_200, sma_slope, high_52w, volume_avg, rs_values):
    price = close[idx]
    
    if np.isnan(sma_200[idx]) or np.isnan(high_52w[idx]) or np.isnan(adx[idx]):
        return False, {}
    
    # 1. Price below 52w high (but not too far)
    pct_below_high = (high_52w[idx] - price) / high_52w[idx] * 100
    if not (MIN_BELOW_HIGH_PCT <= pct_below_high <= MAX_BELOW_HIGH_PCT):
        return False, {}
    
    # 2. Near SMA 200
    pct_from_sma = abs(price - sma_200[idx]) / sma_200[idx] * 100
    if pct_from_sma > MAX_FROM_SMA_PCT:
        return False, {}
    
    # 3. Above SMA 200 (allow 3% below)
    if price < sma_200[idx] * 0.97:
        return False, {}
    
    # 4. SMA slope positive
    if not np.isnan(sma_slope[idx]) and sma_slope[idx] < 0:
        return False, {}
    
    # 5. RSI signal
    rsi_signal = False
    for j in range(1, RSI_LOOKBACK + 1):
        prev_idx, curr_idx = idx - j, idx - j + 1
        if prev_idx >= 0 and curr_idx < len(rsi):
            if rsi[prev_idx] <= RSI_SIGNAL and rsi[curr_idx] > RSI_SIGNAL:
                rsi_signal = True
                break
            if rsi[prev_idx] <= RSI_OVERSOLD:
                rsi_signal = True
                break
    if not rsi_signal:
        return False, {}
    
    # 6. ADX trend
    if adx[idx] < ADX_MIN:
        return False, {}
    
    # 7. Volume confirmation
    vol_ratio = 1.0
    if USE_VOLUME_FILTER:
        if np.isnan(volume_avg[idx]) or volume_avg[idx] == 0:
            return False, {}
        vol_ratio = volume[idx] / volume_avg[idx]
        if vol_ratio < VOLUME_SURGE_MULT:
            return False, {}
    
    # 8. Relative strength
    if USE_RS_FILTER and rs_values is not None:
        if np.isnan(rs_values[idx]) or rs_values[idx] < -0.10:
            return False, {}
    
    details = {
        'pct_below_high': round(pct_below_high, 1),
        'pct_from_sma': round(pct_from_sma, 1),
        'sma_slope': round(sma_slope[idx], 2) if not np.isnan(sma_slope[idx]) else 0,
        'rsi': round(rsi[idx], 1),
        'adx': round(adx[idx], 1),
        'volume_ratio': round(vol_ratio, 2),
    }
    return True, details

print("Signal generation loaded!")

In [ ]:
# Cell 9: Stock Scanner

def scan_stock(ticker, vix_lookup, spy_close):
    df, info = get_stock_data(ticker, START_DATE, END_DATE)
    if df is None or info is None:
        return [], 'no_data'
    
    fund_passed, fund_total = check_fundamentals(info)
    if fund_total > 0 and fund_passed < MIN_FUNDAMENTAL_CHECKS:
        return [], 'bad_fundamentals'
    
    close = df['Close'].values
    high = df['High'].values
    low = df['Low'].values
    volume = df['Volume'].values
    dates = df.index
    
    close_series = pd.Series(close, index=dates)
    high_series = pd.Series(high, index=dates)
    low_series = pd.Series(low, index=dates)
    volume_series = pd.Series(volume, index=dates)
    
    rsi = calc_rsi(close_series, 14).values
    adx, plus_di, minus_di = calc_adx(high_series, low_series, close_series, 14)
    adx, plus_di, minus_di = adx.values, plus_di.values, minus_di.values
    sma_200 = calc_sma(close_series, 200).values
    sma_slope = calc_sma_slope(pd.Series(sma_200), SMA_SLOPE_DAYS).values
    high_52w = high_series.rolling(252).max().values
    volume_avg = volume_series.rolling(VOLUME_AVG_DAYS).mean().values
    
    rs_values = None
    if USE_RS_FILTER and spy_close is not None:
        try:
            aligned_spy = spy_close.reindex(dates).ffill()
            if len(aligned_spy) == len(close):
                rs = calc_relative_strength(close_series, aligned_spy, RS_LOOKBACK)
                rs_values = rs.values
        except:
            pass
    
    signals = []
    last_signal_idx = -999
    start_idx = max(260, SMA_SLOPE_DAYS + 200)
    end_idx = len(df) - MAX_HOLD_DAYS - 5
    
    for i in range(start_idx, end_idx):
        if i - last_signal_idx < MIN_GAP_DAYS:
            continue
        
        date_str = dates[i].strftime('%Y-%m-%d')
        
        if USE_VIX_FILTER:
            vix_value = vix_lookup.get(date_str)
            if vix_value is None or not (VIX_MIN <= vix_value <= VIX_MAX):
                continue
        else:
            vix_value = vix_lookup.get(date_str, 0)
        
        signal_valid, signal_details = check_entry_signal(
            i, close, high, low, volume, rsi, adx, plus_di, minus_di,
            sma_200, sma_slope, high_52w, volume_avg, rs_values
        )
        
        if not signal_valid:
            continue
        
        entry_price = close[i]
        trade_result = execute_trade(close, high, low, i, entry_price)
        
        risk_dollars = ACCOUNT_SIZE * (RISK_PER_TRADE_PCT / 100)
        position_value = min(risk_dollars / (STOP_LOSS_PCT / 100), ACCOUNT_SIZE * (MAX_POSITION_PCT / 100))
        shares = int(position_value / entry_price)
        pnl = position_value * (trade_result['return_pct'] / 100)
        
        signal = {
            'ticker': ticker, 'date': date_str, 'entry': round(entry_price, 2),
            'vix': round(vix_value, 1) if vix_value else None,
            'sector': info.get('sector', 'Unknown'),
            **signal_details,
            'return_pct': round(trade_result['return_pct'], 2),
            'exit_price': round(trade_result['exit_price'], 2),
            'exit_day': trade_result['exit_day'],
            'exit_reason': trade_result['exit_reason'],
            'trailing_activated': trade_result['trailing_activated'],
            'shares': shares, 'position': round(position_value, 0),
            'pnl': round(pnl, 0),
            'max_dd': round(trade_result['max_dd'], 1),
            'max_runup': round(trade_result['max_runup'], 1),
        }
        signals.append(signal)
        last_signal_idx = i
    
    return signals, 'success'

print("Stock scanner loaded!")

In [ ]:
# Cell 10: RUN THE SCREENER

print("="*70)
print("MARKET SNIPER SCREENER v2.0")
print("="*70)

# Download market data
vix_lookup = get_vix_data(START_DATE, END_DATE)
spy_close = get_spy_data(START_DATE, END_DATE) if USE_RS_FILTER else None

# Scan stocks
all_signals = []
stats = {'success': 0, 'no_data': 0, 'bad_fundamentals': 0, 'error': 0}

print(f"\nScanning {len(TICKERS)} stocks...")
start_time = time.time()

for i, ticker in enumerate(TICKERS):
    if (i + 1) % 50 == 0:
        elapsed = time.time() - start_time
        rate = (i + 1) / elapsed * 60 if elapsed > 0 else 0
        eta = (len(TICKERS) - i - 1) / rate if rate > 0 else 0
        print(f"  [{i+1:4d}/{len(TICKERS)}] Signals: {len(all_signals):4d} | {rate:.0f}/min | ETA: {eta:.1f}min")
    
    try:
        signals, status = scan_stock(ticker, vix_lookup, spy_close)
        stats[status] = stats.get(status, 0) + 1
        all_signals.extend(signals)
    except Exception as e:
        stats['error'] += 1
    
    if (i + 1) % 100 == 0:
        time.sleep(1)

elapsed = time.time() - start_time
print(f"\nCompleted in {elapsed/60:.1f} minutes")
print(f"Valid stocks: {stats['success']} | No data: {stats['no_data']} | Bad fundamentals: {stats['bad_fundamentals']}")
print(f"Total signals: {len(all_signals)}")

In [ ]:
# Cell 11: RESULTS ANALYSIS

if all_signals:
    df = pd.DataFrame(all_signals)
    ret = df['return_pct']
    wins = ret > 0
    
    print("="*70)
    print("PERFORMANCE SUMMARY")
    print("="*70)
    print(f"Total Trades:     {len(ret)}")
    print(f"Win Rate:         {wins.mean()*100:.1f}% ({wins.sum()}W / {(~wins).sum()}L)")
    print(f"Avg Return:       {ret.mean():+.2f}%")
    print(f"Avg Win:          +{ret[wins].mean():.2f}%" if wins.any() else "N/A")
    print(f"Avg Loss:         {ret[~wins].mean():.2f}%" if (~wins).any() else "N/A")
    print(f"Best Trade:       +{ret.max():.2f}%")
    print(f"Worst Trade:      {ret.min():.2f}%")
    
    avg_win = ret[wins].mean() if wins.any() else 0
    avg_loss = ret[~wins].mean() if (~wins).any() else 0
    
    print(f"\n{'='*70}")
    print("RISK METRICS")
    print(f"{'='*70}")
    if avg_loss != 0:
        print(f"Win/Loss Ratio:   {abs(avg_win/avg_loss):.2f}:1")
    expectancy = (wins.mean() * avg_win) + ((~wins).mean() * avg_loss)
    print(f"Expectancy:       {expectancy:+.2f}% per trade")
    if (~wins).any() and ret[~wins].sum() != 0:
        pf = ret[wins].sum() / abs(ret[~wins].sum())
        print(f"Profit Factor:    {pf:.2f}")
    if ret.std() > 0:
        print(f"Sharpe-like:      {ret.mean()/ret.std():.3f}")
    
    print(f"\n{'='*70}")
    print("DOLLAR P&L")
    print(f"{'='*70}")
    print(f"Total P&L:        ${df['pnl'].sum():+,.0f}")
    print(f"Avg P&L/Trade:    ${df['pnl'].mean():+,.0f}")
    print(f"Avg Hold:         {df['exit_day'].mean():.1f} days")
    
    print(f"\n{'='*70}")
    print("EXIT BREAKDOWN")
    print(f"{'='*70}")
    for reason in ['stop', 'trail_stop', 'target', 'max_days']:
        subset = df[df['exit_reason'] == reason]
        if len(subset) > 0:
            pct = len(subset) / len(df) * 100
            avg_ret = subset['return_pct'].mean()
            print(f"  {reason:12s}: {len(subset):4d} ({pct:5.1f}%) | Avg: {avg_ret:+6.2f}%")
    
    trail_count = df['trailing_activated'].sum()
    print(f"\nTrailing Activated: {trail_count}/{len(df)} ({trail_count/len(df)*100:.1f}%)")
else:
    print("No signals found. Try adjusting filters.")
    df = pd.DataFrame()

In [ ]:
# Cell 12: BY YEAR ANALYSIS

if len(df) > 0:
    df['year'] = pd.to_datetime(df['date']).dt.year
    print("PERFORMANCE BY YEAR")
    print("="*70)
    yearly = df.groupby('year').agg({
        'return_pct': ['count', 'mean', lambda x: (x > 0).mean() * 100],
        'pnl': 'sum'
    }).round(2)
    yearly.columns = ['Trades', 'Avg%', 'WinRate%', 'TotalPnL']
    print(yearly.to_string())

In [ ]:
# Cell 13: BY SECTOR ANALYSIS

if len(df) > 0:
    print("PERFORMANCE BY SECTOR")
    print("="*70)
    sector = df.groupby('sector').agg({
        'return_pct': ['count', 'mean', lambda x: (x > 0).mean() * 100],
        'pnl': 'sum'
    }).round(2)
    sector.columns = ['Trades', 'Avg%', 'WinRate%', 'TotalPnL']
    sector = sector.sort_values('WinRate%', ascending=False)
    print(sector.to_string())

In [ ]:
# Cell 14: TOP & BOTTOM TRADES

if len(df) > 0:
    cols = ['ticker', 'date', 'return_pct', 'exit_reason', 'exit_day', 'pnl']
    
    print("TOP 15 TRADES")
    print("="*70)
    print(df.nlargest(15, 'return_pct')[cols].to_string(index=False))
    
    print(f"\nBOTTOM 10 TRADES")
    print("="*70)
    print(df.nsmallest(10, 'return_pct')[cols].to_string(index=False))

In [ ]:
# Cell 15: SAVE & DOWNLOAD RESULTS

if len(df) > 0:
    output_file = 'screener_results.csv'
    df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    
    # Download in Colab
    try:
        from google.colab import files
        files.download(output_file)
        print("Download started!")
    except ImportError:
        print("(Not in Colab - file saved locally)")